In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkarsabnis/yelp-reviews-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.49M/3.49M [00:00<00:00, 102MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/omkarsabnis/yelp-reviews-dataset/versions/1


In [4]:
import os

os.listdir(path)


['yelp.csv']

In [6]:
import pandas as pd

csv_path = os.path.join(path, "yelp.csv")
df = pd.read_csv(csv_path)

df.head()


,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [8]:
df.columns



Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')

In [9]:
df = df.rename(columns={
    "review": "text",
    "rating": "stars"
})


In [36]:
df = df[['text', 'stars']]   # keep only what we need
df_sample = df.sample(30, random_state=42).reset_index(drop=True)

df_sample.head()


,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5


In [37]:
df_sample['stars'].value_counts().sort_index()


,count
stars,
1,4
2,3
3,2
4,12
5,9


In [38]:
def call_llm(prompt):
    return """
    {
      "predicted_stars": 4,
      "explanation": "The review is mostly positive with a minor complaint about service speed."
    }
    """


In [39]:
print(call_llm("Classify this review: Great food but slow service"))



    {
      "predicted_stars": 4,
      "explanation": "The review is mostly positive with a minor complaint about service speed."
    }
    


In [16]:
!pip install -q google-generativeai


In [22]:
import os
import getpass

os.environ["GEMINI_API_KEY"] = getpass.getpass("type your API key")


type your API key··········


In [33]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-pro-latest",  # Changed model name to gemini-pro-latest
    generation_config={
        "temperature": 0.2,          # lower = more consistent
        "max_output_tokens": 200
    }
)

In [25]:
def call_llm(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"ERROR: {str(e)}"


In [41]:
test_prompt = """
You are an AI that classifies Yelp reviews.

Review:
"Great food but slow service."

Return ONLY valid JSON:
{
  "predicted_stars": <1-5>,
  "explanation": "<brief reason>"
}
"""

print(call_llm(test_prompt))



    {
      "predicted_stars": 4,
      "explanation": "The review is mostly positive with a minor complaint about service speed."
    }
    


In [44]:
import time
from tqdm import tqdm

def evaluate_prompt(prompt_template):
    results = []

    for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
        prompt = prompt_template.format(review=row['text'])
        response = call_llm(prompt)
        parsed = extract_json(response)

        results.append({
            "actual_stars": row['stars'],
            "response": response,
            "parsed": parsed
        })

        time.sleep(3)  # IMPORTANT for Gemini free tier

    return pd.DataFrame(results)


In [46]:
print(evaluate_prompt)


<function evaluate_prompt at 0x7df5b8bd59e0>


In [52]:

zero_shot_prompt = """
You are an AI that classifies Yelp reviews into star ratings from 1 to 5.

Review:
"{review}"

Return ONLY valid JSON in this format:
{{
  "predicted_stars": <integer from 1 to 5>,
  "explanation": "<brief reasoning>"
}}
"""


In [53]:
few_shot_prompt = """
You are an AI that assigns Yelp star ratings.

Examples:

Review: "Terrible service. Food was cold and staff rude."
Output:
{{
  "predicted_stars": 1,
  "explanation": "Strong negative sentiment and poor service."
}}

Review: "Amazing food and friendly staff. Loved the experience."
Output:
{{
  "predicted_stars": 5,
  "explanation": "Very positive sentiment throughout."
}}

Now classify this review:

Review:
"{review}"

Return ONLY valid JSON:
{{
  "predicted_stars": <1-5>,
  "explanation": "<brief reasoning>"
}}
"""


In [50]:
print(structured_prompt[:200])



You are a strict classifier.

Rating rubric:
1 = Very negative, strong complaints
2 = Mostly negative
3 = Mixed or neutral
4 = Mostly positive with minor issues
5 = Extremely positive

Rules:
- Outpu


In [54]:
structured_prompt = """
You are a strict classifier.

Rating rubric:
1 = Very negative, strong complaints
2 = Mostly negative
3 = Mixed or neutral
4 = Mostly positive with minor issues
5 = Extremely positive

Rules:
- Output MUST be valid JSON
- predicted_stars must be an integer between 1 and 5
- No extra text outside JSON

Review:
"{review}"

JSON output only:
{{
  "predicted_stars": <1-5>,
  "explanation": "<short reason>"
}}
"""


In [55]:
print(structured_prompt.format(review="Great food but slow service"))



You are a strict classifier.

Rating rubric:
1 = Very negative, strong complaints
2 = Mostly negative
3 = Mixed or neutral
4 = Mostly positive with minor issues
5 = Extremely positive

Rules:
- Output MUST be valid JSON
- predicted_stars must be an integer between 1 and 5
- No extra text outside JSON

Review:
"Great food but slow service"

JSON output only:
{
  "predicted_stars": <1-5>,
  "explanation": "<short reason>"
}



In [57]:
import json
import re

def extract_json(text):
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
        return None
    except Exception:
        return None


In [58]:
df_structured = evaluate_prompt(structured_prompt)
df_structured.head()


100%|██████████| 30/30 [01:30<00:00,  3.00s/it]


,actual_stars,response,parsed
0,4,"\n {\n ""predicted_stars"": 4,\n ""e...","{'predicted_stars': 4, 'explanation': 'The rev..."
1,5,"\n {\n ""predicted_stars"": 4,\n ""e...","{'predicted_stars': 4, 'explanation': 'The rev..."
2,3,"\n {\n ""predicted_stars"": 4,\n ""e...","{'predicted_stars': 4, 'explanation': 'The rev..."
3,1,"\n {\n ""predicted_stars"": 4,\n ""e...","{'predicted_stars': 4, 'explanation': 'The rev..."
4,5,"\n {\n ""predicted_stars"": 4,\n ""e...","{'predicted_stars': 4, 'explanation': 'The rev..."


In [59]:
df_zero = evaluate_prompt(zero_shot_prompt)
df_few = evaluate_prompt(few_shot_prompt)


100%|██████████| 30/30 [01:30<00:00,  3.00s/it]


In [60]:
def compute_metrics(df):
    valid = df['parsed'].notnull()
    df_valid = df[valid].copy()

    df_valid['predicted'] = df_valid['parsed'].apply(lambda x: x['predicted_stars'])

    accuracy = (df_valid['predicted'] == df_valid['actual_stars']).mean()
    json_validity = valid.mean()

    return accuracy, json_validity


In [61]:
metrics = pd.DataFrame([
    ("Zero-shot", *compute_metrics(df_zero)),
    ("Few-shot", *compute_metrics(df_few)),
    ("Structured", *compute_metrics(df_structured)),
], columns=["Prompt Type", "Accuracy", "JSON Validity"])

metrics


,Prompt Type,Accuracy,JSON Validity
0,Zero-shot,0.4,1.0
1,Few-shot,0.4,1.0
2,Structured,0.4,1.0


In [62]:
def reliability_test(review, prompt, runs=5):
    outputs = []
    for _ in range(runs):
        response = call_llm(prompt.format(review=review))
        parsed = extract_json(response)
        outputs.append(parsed["predicted_stars"] if parsed else None)
    return outputs

test_review = df_sample.iloc[0]['text']
reliability_test(test_review, structured_prompt)


[4, 4, 4, 4, 4]

In [63]:
metrics.to_csv("task1_metrics.csv", index=False)
df_structured.to_csv("structured_outputs.csv", index=False)
